## Exploration_09_영화추천
1. 파일 불러오기, 전처리
2. CSR matrix 형성
3. MF model 훈련
4. 추천영화 확인
5. 회고

### Step 1. 파일 불러오기  & 데이터 전처리

In [63]:
import os
import pandas as pd

rating_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python', encoding = "ISO-8859-1")
orginal_data_size = len(ratings)
ratings.head()

,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [64]:
# 3점 이상만 남깁니다.
ratings = ratings[ratings['rating']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [65]:
# rating 컬럼의 이름을 counts로 바꿉니다.
ratings.rename(columns={'rating':'counts'}, inplace=True)

In [66]:
ratings['counts']

0          5
1          3
2          3
3          4
4          5
          ..
1000203    3
1000205    5
1000206    5
1000207    4
1000208    4
Name: counts, Length: 836478, dtype: int64

* column 이름을 'count'로 할 경우 뒤에서 dataframe 자체 method인 'count'가 호출 되기 때문에 'counts'로 변경한다.

In [67]:
# 영화 제목을 보기 위해 메타 데이터를 읽어옵니다.
movie_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python', encoding='ISO-8859-1')
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [68]:
# title, genre 소문자로 변경
movies['title'] = movies['title'].str.lower() 
movies['genre'] = movies['genre'].str.lower() 
movies.head(10)

,movie_id,title,genre
0,1,toy story (1995),animation|children's|comedy
1,2,jumanji (1995),adventure|children's|fantasy
2,3,grumpier old men (1995),comedy|romance
3,4,waiting to exhale (1995),comedy|drama
4,5,father of the bride part ii (1995),comedy
5,6,heat (1995),action|crime|thriller
6,7,sabrina (1995),comedy|romance
7,8,tom and huck (1995),adventure|children's
8,9,sudden death (1995),action
9,10,goldeneye (1995),action|adventure|thriller


* 소문자 처리 등 간단한 전처리들을 완료

In [69]:
# movies 와 ratings merge
movies_mer = pd.merge(movies, ratings, how='inner', on='movie_id')
movies_mer.drop(['timestamp'], axis=1, inplace=True)

In [70]:
movies_mer.head()

,movie_id,title,genre,user_id,counts
0,1,toy story (1995),animation|children's|comedy,1,5
1,1,toy story (1995),animation|children's|comedy,6,4
2,1,toy story (1995),animation|children's|comedy,8,4
3,1,toy story (1995),animation|children's|comedy,9,5
4,1,toy story (1995),animation|children's|comedy,10,5


* 좀더 간단화 하기 위해 'movies', 'ratings' dataframe을 'movie_id'를 기준으로 합친다.

In [71]:
# 유저 수
movies_mer['user_id'].nunique()

6039

In [72]:
# 영화 수
movies['title'].nunique()

3883

In [73]:
# 인기 많은 영화
movie_count = movies_mer.groupby('title')['user_id'].count()
movie_count.sort_values(ascending=False).head(10)

title
american beauty (1999)                                   3211
star wars: episode iv - a new hope (1977)                2910
star wars: episode v - the empire strikes back (1980)    2885
star wars: episode vi - return of the jedi (1983)        2716
saving private ryan (1998)                               2561
terminator 2: judgment day (1991)                        2509
silence of the lambs, the (1991)                         2498
raiders of the lost ark (1981)                           2473
back to the future (1985)                                2460
matrix, the (1999)                                       2434
Name: user_id, dtype: int64

In [74]:
# 유저별 몇 개의 영화를 보고 있는지
user_count = movies_mer.groupby('user_id')['title'].count()
user_count.describe()

count    6039.000000
mean      138.512668
std       156.241599
min         1.000000
25%        38.000000
50%        81.000000
75%       177.000000
max      1968.000000
Name: title, dtype: float64

* 유저 수, 영화 수 등의 정보에 대한 수치를 직접 확인

In [75]:
# 본인이 좋아하시는 아티스트 데이터로 바꿔서 추가하셔도 됩니다! 단, 이름은 꼭 데이터셋에 있는 것과 동일하게 맞춰주세요. 
my_favorite = ['toy story (1995)' , 'jumanji (1995)' ,'sabrina (1995)', 'Father of the Bride Part II (1995)', 'grumpier old men (1995)']
my_genre = ['animation|children\'s|comedy', 'Adventure|Children\'s|Fantasy', 'comedy|romance', 'comedy', 'comedy|romance']

# 'zimin'이라는 user_id가 위 아티스트의 노래를 30회씩 들었다고 가정하겠습니다.
my_movielist = pd.DataFrame({'user_id': ['jason']*5, 'title': my_favorite, 'genre': my_genre, 'movie_id': [1, 2, 7, 5, 3], 'counts':[4, 5, 4, 4, 5]})

if not movies_mer.isin({'user_id': ['jason']})['user_id'].any():  # 본인 user_id 를 '9999' 로 임시 설정
    movies_mer = movies_mer.append(my_movielist) 

movies_mer.tail(10)       # 잘 추가되었는지 확인해 봅시다.

,movie_id,title,genre,user_id,counts
836473,3952,"contender, the (2000)",drama|thriller,5682,3
836474,3952,"contender, the (2000)",drama|thriller,5812,4
836475,3952,"contender, the (2000)",drama|thriller,5831,3
836476,3952,"contender, the (2000)",drama|thriller,5837,4
836477,3952,"contender, the (2000)",drama|thriller,5998,4
0,1,toy story (1995),animation|children's|comedy,jason,4
1,2,jumanji (1995),Adventure|Children's|Fantasy,jason,5
2,7,sabrina (1995),comedy|romance,jason,4
3,5,Father of the Bride Part II (1995),comedy,jason,4
4,3,grumpier old men (1995),comedy|romance,jason,5


In [76]:
# 고유한 영화제목, 장르를 찾아내는 코드
genre_unique = movies_mer['genre'].unique()
title_unique = movies_mer['title'].unique()
id_unique = movies_mer['user_id'].unique()

genre_to_idx = {v:k for k,v in enumerate(genre_unique)}
title_to_idx = {v:k for k,v in enumerate(title_unique)}
id_to_idx = {v:k for k,v in enumerate(id_unique)}

In [77]:
# 인덱싱이 잘 되었는지 확인해 봅니다. 
print(genre_to_idx['comedy'])
print(title_to_idx['jumanji (1995)'])
print(id_to_idx['jason'])

4
1
6039


In [78]:
# indexing을 통해 데이터 컬럼 내 값을 바꾸는 코드
# dictionary 자료형의 get 함수는 https://wikidocs.net/16 을 참고하세요.

# user_id to index
temp_id_data = movies_mer['user_id'].map(id_to_idx.get).dropna() # 변환 되지 않은 NaN 제거
if len(temp_id_data) == len(movies_mer):   # 모든 row가 정상적으로 인덱싱되었다면
    print('user_id column indexing OK!!')
    movies_mer['user_id'] = temp_id_data   # indexing 된 genre data로 입력
else:
    print('user_id column indexing Fail!!')

# genre to index
temp_genre_data = movies_mer['genre'].map(genre_to_idx.get).dropna() # 변환 되지 않은 NaN 제거
if len(temp_genre_data) == len(movies_mer):   # 모든 row가 정상적으로 인덱싱되었다면
    print('genre column indexing OK!!')
    movies_mer['genre'] = temp_genre_data   # indexing 된 genre data로 입력
else:
    print('genre column indexing Fail!!')

# title to index
temp_title_data = movies_mer['title'].map(title_to_idx.get).dropna()
if len(temp_title_data) == len(movies_mer):
    print('title column indexing OK!!')
    movies_mer['title'] = temp_title_data
else:
    print('title column indexing Fail!!')

movies_mer

user_id column indexing OK!!
genre column indexing OK!!
title column indexing OK!!


,movie_id,title,genre,user_id,counts
0,1,0,0,0,5
1,1,0,0,1,4
2,1,0,0,2,4
3,1,0,0,3,5
4,1,0,0,4,5
...,...,...,...,...,...
0,1,0,0,6039,4
1,2,1,301,6039,5
2,7,6,2,6039,4
3,5,3628,4,6039,4


### Step 2. CSR matrix 형성

In [79]:
# to CSR matrix
from scipy.sparse import csr_matrix

num_user = movies_mer['user_id'].nunique()
num_title = movies_mer['title'].nunique()

csr_data = csr_matrix((movies_mer.counts, (movies_mer.user_id, movies_mer.title)), shape=(num_user, num_title))
csr_data

<6040x3629 sparse matrix of type '<class 'numpy.longlong'>'
	with 836483 stored elements in Compressed Sparse Row format>

* 위에서 설명한 대로 'movies_mer.count' 를 사용시 count method를 호출하기 때문에 'counts'로 column 명을 바꿔주었다.

### Step 3. Model Fitting

In [80]:
from implicit.als import AlternatingLeastSquares
import os
import numpy as np

# implicit 라이브러리에서 권장하고 있는 부분입니다. 학습 내용과는 무관합니다.
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

# Implicit AlternatingLeastSquares 모델의 선언
als_model = AlternatingLeastSquares(factors=100, regularization=0.01, use_gpu=False, iterations=15, dtype=np.float32)

In [81]:
# als 모델은 input으로 (item X user 꼴의 matrix를 받기 때문에 Transpose해줍니다.)
csr_data_transpose = csr_data.T
csr_data_transpose

<3629x6040 sparse matrix of type '<class 'numpy.longlong'>'
	with 836483 stored elements in Compressed Sparse Column format>

In [82]:
# 모델 훈련
als_model.fit(csr_data_transpose)

  0%|          | 0/15 [00:00<?, ?it/s]

In [83]:
jason, toy_story = id_to_idx['jason'], title_to_idx['toy story (1995)']
jason_vector, toy_story_vector = als_model.user_factors[jason], als_model.item_factors[toy_story]

print('Done')

Done


In [84]:
jason_vector

array([ 0.68963206,  0.05963608,  0.1359798 ,  0.44446045,  0.39936262,
       -0.16899644, -0.04499798,  1.0706142 , -0.6229414 ,  0.15867403,
       -0.63311195, -0.18573625,  0.7607227 , -0.2575649 ,  0.04069348,
        0.2214195 ,  0.47968552, -0.04893998,  0.2962481 , -0.2482972 ,
       -0.22799024,  0.08100133, -0.26683727, -0.18607163, -0.19530055,
       -0.28465608, -0.22816858,  0.08700449,  0.5151899 , -0.5024421 ,
        0.07418741, -0.29951644, -0.25652447, -0.5737436 ,  0.32531226,
        0.5963507 , -0.14890464, -0.19199914, -0.05773946,  0.15742846,
        0.219191  ,  0.4785472 ,  0.49645463,  0.20399366, -0.4184186 ,
       -0.12576488, -0.3948794 ,  0.927468  , -0.4948874 , -0.27289513,
       -0.63683033, -0.10821484, -0.15749149,  0.22186464,  0.13578814,
        0.22762264,  0.23794511,  0.11025166,  0.7154567 ,  0.11926684,
        0.16099125,  0.40636355,  0.2907196 ,  0.06150615,  0.02852562,
       -0.03093946,  0.2676144 , -0.2701607 , -0.27308503, -0.01

In [85]:
toy_story_vector

array([ 0.03328296,  0.00751261,  0.0226194 ,  0.00396802,  0.02972207,
        0.00100285, -0.01581656,  0.03153069, -0.02892608,  0.0156072 ,
       -0.00338922, -0.00268193,  0.01664031, -0.01304582, -0.01840759,
        0.00552248,  0.03273223, -0.01864478, -0.00351506,  0.00555818,
       -0.01696168,  0.01568446, -0.02597936, -0.01026447, -0.04129355,
       -0.00118053,  0.0217074 , -0.00838505,  0.02583338, -0.02673255,
       -0.00503372,  0.0083062 ,  0.01925418, -0.01099021, -0.00754062,
        0.05192791, -0.00613574,  0.00015603,  0.02111957,  0.01839323,
        0.01044573,  0.008836  ,  0.01451371, -0.00982899, -0.0128245 ,
        0.00421465, -0.01269462,  0.02626009, -0.0252354 ,  0.00606487,
        0.01974913,  0.0138822 , -0.00425232,  0.01091912, -0.00401266,
        0.00815268, -0.01022989,  0.00022818,  0.00633867,  0.0150246 ,
        0.00780436,  0.01471304,  0.00490452,  0.02964201, -0.00023032,
        0.00698937,  0.00967686,  0.00720331, -0.01118897,  0.02

In [86]:
# jason, toy_story 의 내적
np.dot(jason_vector, toy_story_vector)

0.387409

* parameter 수정이 필요해 보인다. 1에 가까운 값이 나올 수록 학습이 잘 된 상태이다.

In [87]:
# factors : dimensions
# iterations : epochs
als_model = AlternatingLeastSquares(factors=300, regularization=0.01, use_gpu=False,\
                                    iterations=100, dtype=np.float32, random_state=1)

In [88]:
als_model.fit(csr_data_transpose)

  0%|          | 0/100 [00:00<?, ?it/s]

In [89]:
# vector
jason, toy_story = id_to_idx['jason'], title_to_idx['toy story (1995)']
jason_vector, toy_story_vector = als_model.user_factors[jason], als_model.item_factors[toy_story]
# 내적
np.dot(jason_vector, toy_story_vector)

0.81195277

* 위와 같이 parameter를 수정하여 학습 정확도를 두 배 이상 높였다.
* facotrs = 100 -> 300 / iterations = 15 -> 100

### Step 4. 추천 영화 확인
#### 임의의 영화에 대한 추천 확인

In [90]:
# 좋아하는 영화와 비슷한 영화 추천
favorite_movie = 'heat (1995)'
movie_id = title_to_idx[favorite_movie]
similar_artist = als_model.similar_items(movie_id, N=15)
similar_artist

[(5, 1.0),
 (2040, 0.32934695),
 (536, 0.29997572),
 (456, 0.22667809),
 (475, 0.22064647),
 (1805, 0.2097218),
 (615, 0.19549933),
 (2841, 0.19296962),
 (1694, 0.19289695),
 (1838, 0.18993272),
 (612, 0.18811993),
 (2150, 0.1876023),
 (1995, 0.18596213),
 (413, 0.18368265),
 (117, 0.1836744)]

In [91]:
#artist_to_idx 를 뒤집어, index로부터 artist 이름을 얻는 dict를 생성합니다. 
idx_to_movie = {v:k for k,v in title_to_idx.items()}
[idx_to_movie[i[0]] for i in similar_artist]

['heat (1995)',
 'ronin (1998)',
 'true romance (1993)',
 'in the line of fire (1993)',
 'menace ii society (1993)',
 'godfather: part iii, the (1990)',
 'peanuts - die bank zahlt alles (1996)',
 'country (1984)',
 'out of sight (1998)',
 'negotiator, the (1998)',
 'jack and sarah (1995)',
 'simple plan, a (1998)',
 'blackmail (1929)',
 "carlito's way (1993)",
 'boomerang (1992)']

In [92]:
def get_similar_movie(movie_name: str):
    movie_id = title_to_idx[movie_name]
    similar_artist = als_model.similar_items(movie_id)
    similar_artist = [idx_to_movie[i[0]] for i in similar_artist]
    return similar_artist

print('Done!')

Done!


In [93]:
get_similar_movie('heat (1995)')

['heat (1995)',
 'ronin (1998)',
 'true romance (1993)',
 'in the line of fire (1993)',
 'menace ii society (1993)',
 'godfather: part iii, the (1990)',
 'peanuts - die bank zahlt alles (1996)',
 'country (1984)',
 'out of sight (1998)',
 'negotiator, the (1998)']

* 함수가 제대로 작동하는지 보기 위해 함수로 만들기 전과 후의 결과를 비교해 봤더니 동일하게 작동하는 것을 알 수 있었다.

#### 사용자에 따른 추천 영화 확인

In [94]:
user = id_to_idx['jason']
# recommend에서는 user*item CSR Matrix를 받습니다.
movie_recommended = als_model.recommend(user, csr_data, N=20, filter_already_liked_items=True)
movie_recommended

[(3151, 0.35236084),
 (2845, 0.27510774),
 (521, 0.18542396),
 (58, 0.18290064),
 (622, 0.17402616),
 (2183, 0.1576846),
 (1391, 0.15252556),
 (1677, 0.14521061),
 (687, 0.14075509),
 (985, 0.13176525),
 (1266, 0.13093568),
 (2506, 0.1288254),
 (235, 0.12857153),
 (1954, 0.12855129),
 (1041, 0.12765513),
 (241, 0.1259302),
 (3187, 0.124969155),
 (450, 0.11981725),
 (1534, 0.11913767),
 (102, 0.118276246)]

* 추천율이 높은 영화들을 가져온다

In [95]:
# 추천 영화 목록
rec_movie = [idx_to_movie[i[0]] for i in movie_recommended]

In [96]:
rec_movie

['grumpy old men (1993)',
 'toy story 2 (1999)',
 'sleepless in seattle (1993)',
 'indian in the cupboard, the (1995)',
 'dragonheart (1996)',
 "you've got mail (1998)",
 'lost world: jurassic park, the (1997)',
 'six days seven nights (1998)',
 'twister (1996)',
 'fish called wanda, a (1988)',
 'jerry maguire (1996)',
 'iron giant, the (1999)',
 'hoop dreams (1994)',
 'beetlejuice (1988)',
 'wrong trousers, the (1993)',
 'i.q. (1994)',
 'hook (1991)',
 'englishman who went up a hill, but came down a mountain, the (1995)',
 'midnight in the garden of good and evil (1997)',
 'bridges of madison county, the (1995)']

In [97]:
# 추천영화의 index 추출
rec_movie_idx = [movies.index[movies['title'] == i] for i in rec_movie]

In [98]:
# 추천 영화의 장르 추출
[movies.loc[i]['genre'] for i in rec_movie_idx]

# 위에서 입력한 본인이 좋아하는 영화 목록의 장르와 비교
# my_favorite = ['toy story (1995)' , 'jumanji (1995)' ,'sabrina (1995)', 'Father of the Bride Part II (1995)', 'grumpier old men (1995)']
# my_genre = ['animation|children\'s|comedy', 'Adventure|Children\'s|Fantasy', 'comedy|romance', 'comedy', 'comedy|romance']

[3381    comedy
 Name: genre, dtype: object,
 3045    animation|children's|comedy
 Name: genre, dtype: object,
 535    comedy|romance
 Name: genre, dtype: object,
 59    adventure|children's|fantasy
 Name: genre, dtype: object,
 647    action|adventure|fantasy
 Name: genre, dtype: object,
 2355    comedy|romance
 Name: genre, dtype: object,
 1505    action|adventure|sci-fi|thriller
 Name: genre, dtype: object,
 1825    adventure|comedy|romance
 Name: genre, dtype: object,
 727    action|adventure|romance|thriller
 Name: genre, dtype: object,
 1063    comedy
 Name: genre, dtype: object,
 1372    drama|romance
 Name: genre, dtype: object,
 2692    animation|children's
 Name: genre, dtype: object,
 243    documentary
 Name: genre, dtype: object,
 2105    comedy|fantasy
 Name: genre, dtype: object,
 1132    animation|comedy
 Name: genre, dtype: object,
 249    comedy|romance
 Name: genre, dtype: object,
 3420    adventure|fantasy
 Name: genre, dtype: object,
 464    comedy|romance
 Name: g

* 추천을 받기위한 기준으로 입력한 본인이 좋아하는 영화들의 장르를 살펴보면 'comedy', 'romance', 'fantasy' 등이 메인이라고 할 수 있다. 이와 비교해서 추천된 영화들의 장르를 보면 마찬가지로 'comedy', 'romance', 'fantasy' 등이 주된 내용인 영화들이라고 볼 수 있고, 'drama', 'thriller', 'documentary' 등의 장르들은 상대적으로 추천이 많지 않은 것을 알 수 있다.
* 따라서 결론적으로 어느정도 사용자의 성향에 맞춘 의미있는 추천이 이뤄졌다고 할 수 있다.

In [99]:
# toy_story2 영화가 추천 된 이유 확인
toy_story2= title_to_idx['toy story 2 (1999)']
explain = als_model.explain(user, csr_data, itemid=toy_story2)

In [100]:
# 추쳔영화에 가장 영향을 많이 준 순서
[(idx_to_movie[i[0]], i[1]) for i in explain[1]]

[('toy story (1995)', 0.23065821078414503),
 ('grumpier old men (1995)', 0.03669648815159135),
 ('sabrina (1995)', 0.027245158770180355),
 ('Father of the Bride Part II (1995)', 0.0007928889363428096),
 ('jumanji (1995)', -0.02096343228893597)]

### Step 5. 회고
* 처음 노드를 접했을 때 이해가 잘 되지 않았던 부분은 CSR matrix를 형성하는 방식이었다. CSR matrix를 사용하면 메모리사용량이나 계산 속도면에서 이점이 있다고 하지만, 너무 sparse한 형태의 matrix는 오히려 accuracy를 낮추는 문제가 생길 수 도 있다고 한다. 따라서 상황에 맞춰 사용하는 것이 필요해 보인다. 
* 추천이 정상적으로 이뤄졌는지 확인하기 위해, 기준이 되는 내가 좋아하는 영화들의 장르의 메인을 comedy와 fantasy로 정하고 진행해 보았다. 영화에 따라 여러 장르를 포함하기도 했기에 최종 결과값에서 다양한 장르의 영화들이 추천 되었지만 결과적으로 comedy와 fantasy, romance 장르가 포함된 영화들이 다수 추천 된 것을 확인할 수 있었다. 따라서 어느정도 사용자의 성향에 맞춘 적절한 수준의 추천이 이뤄졌다고 생각한다.
* 추천영화의 기여도를 확인해 보면 toy story2가 추천 된 이유에 대해 toy story가 가장 큰 기여도를 가지는 걸로 봐서 이 또한 적절하게 이뤄졌다고 볼 수 있다.